In [ ]:
# Imports
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import mne
from mne.stats import combine_adjacency, f_mway_rm, f_threshold_mway_rm, spatio_temporal_cluster_test

In [ ]:
# Configuration
HEMI = 'lh'
USE_ROI = True
SEARCH_TMIN = 0
SEARCH_TMAX = 400

ROOT_DIR = 'data'
SUBJECTS_DIR = join(ROOT_DIR, 'mri')
STC_DIR = join(ROOT_DIR, 'stc')
STATS_DIR = join(ROOT_DIR, 'stats')

subjects = [f'subject_{i:03d}' for i in range(1, 21)]
brodmann_areas = ['20', '21', '22', '38', '39', '11', '44', '45']

condition_colors = {
    'condition_1': 'tab:blue',
    'condition_2': 'tab:orange', 
    'condition_3': 'tab:green',
    'condition_4': 'tab:red'
}
conditions = list(condition_colors.keys())

In [ ]:
# Helper functions
def get_stc(subject_id, condition):
    stc_fname = os.path.join(STC_DIR, condition, f'{subject_id}_{condition}_dSPM')
    stc = mne.read_source_estimate(stc_fname, subject='fsaverage')
    return stc

def stat_fun(*args):
    return f_mway_rm(
        np.swapaxes(args, 1, 0), 
        factor_levels=factor_levels,
        effects=effects, 
        return_pvals=True
    )[0]

In [ ]:
# Setup source space and ROI
src_fname = os.path.join(SUBJECTS_DIR, 'fsaverage', 'bem', 'fsaverage-ico-4-src.fif')
src = mne.read_source_spaces(src_fname)

# Define ROI (Brodmann area example)
brodmann_num = brodmann_areas[0]
annot_name = 'PALS_B12_Brodmann'
brodmann = mne.read_labels_from_annot(
    'fsaverage', annot_name, 
    subjects_dir=SUBJECTS_DIR, 
    hemi=HEMI
)

label_name = f'Brodmann.{brodmann_num}-{HEMI}'
roi = [label for label in brodmann if label.name == label_name][0]

# Get ROI indices
n_hemisources = 2562
hemi_idx = np.arange(0, n_hemisources, 1)
roi_idx = roi.get_vertices_used(vertices=hemi_idx)
diff_idx = np.setdiff1d(hemi_idx, roi_idx)
spatial_exclude = diff_idx if USE_ROI else None

In [ ]:
# Visualize parcellation with fsaverage
Brain = mne.viz.get_brain_class()

brain = Brain(
    "fsaverage",
    HEMI,
    "inflated",  
    subjects_dir=SUBJECTS_DIR,
    cortex="low_contrast",
    background="white",
    size=(800, 600),
    views=['lateral']
)
        
brain.add_label(roi, borders=False, color='tab:purple')

In [ ]:
# Load and prepare data
data_time = []
times = np.arange(-100, 801, 1)

for c, condition in enumerate(conditions):
    n_subj = len(subjects)
    data_tmp = np.empty((n_subj, n_hemisources, len(times)))
    print(f'Reading STCs for {condition}')
    
    for s, subject_id in enumerate(subjects):
        stc = get_stc(subject_id, condition)
        data_tmp[s, :, :] = stc.data[:n_hemisources]
    
    data_transposed = np.transpose(data_tmp, [0, 2, 1])
    
    tmin_idx = np.where(times == SEARCH_TMIN)[0][0]
    tmax_idx = np.where(times == SEARCH_TMAX)[0][0]
    toi_idx = np.arange(tmin_idx, tmax_idx + 1)
    
    data_time.append(data_transposed[:, toi_idx, :n_hemisources])

print(f"Time window: {SEARCH_TMIN} to {SEARCH_TMAX} ms")

In [ ]:
# Statistical test setup
tail = 0
p_thresh = 0.05
n_permutations = 1000

factor_levels = [2, 2]  # 2x2 ANOVA
effects = 'A'  # Main effect A
f_thresh = f_threshold_mway_rm(len(subjects), factor_levels, effects, p_thresh)

# Setup adjacency
if HEMI == 'lh':
    adjacency = mne.spatial_src_adjacency(src[:1])
elif HEMI == 'rh':
    adjacency = mne.spatial_src_adjacency(src[1:])
else:
    adjacency = mne.spatial_src_adjacency(src)

print(f"ROI: {roi.name}, vertices: {len(roi_idx)}" if USE_ROI else f"Use ROI: {USE_ROI}")
print(f"Data shape: {data_time[0].shape}")
print(f"F threshold: {f_thresh:.3f}")

In [ ]:
# Run cluster test
F_obs, clusters, clusters_pvals, h0 = mne.stats.spatio_temporal_cluster_test(
    data_time,
    tail=tail,
    threshold=f_thresh,
    stat_fun=stat_fun,
    n_permutations=n_permutations,
    adjacency=adjacency,
    spatial_exclude=spatial_exclude,
    out_type='indices',
    n_jobs=4
)

# Save results
pickle_fname = os.path.join(
    STATS_DIR, 
    f'stats_{effects}_{SEARCH_TMIN}-{SEARCH_TMAX}_{label_name}.pkl'
)
os.makedirs(STATS_DIR, exist_ok=True)

with open(pickle_fname, "wb") as f:
    pickle.dump((F_obs, clusters, clusters_pvals, h0), f)

print(f"Cluster p-values: {clusters_pvals}")
significant_clusters = np.where(clusters_pvals < 0.05)[0]
print(f"Significant clusters: {significant_clusters}")